<a href="https://colab.research.google.com/github/Snake-AID/SnakeAid.ComputerVision/blob/main/SnakeAid_Training_YOLOv12_Bbox5000_Khiem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment setup

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Define Dataset Paths

After mounting Google Drive, you need to define `DATASET_PATH` to point to the root directory of your dataset and `DATA_YAML_PATH` to point to your `data.yaml` file. Please replace the example paths with your actual paths on Google Drive.

**Example structure on Google Drive:**
```
My Drive/
├── my_yolo_dataset/
│   ├── train/
│   ├── valid/
│   ├── test/
│   └── data.yaml
```

In [ ]:
DATASET_PATH = '/content/drive/MyDrive/SnakeDataset/SnakeAid-5000bbox.yolov12'
DATA_YAML_PATH = f'{DATASET_PATH}/data.yaml'

In [ ]:
print(f"DATASET_PATH: {DATASET_PATH}")
print(f"DATA_YAML_PATH: {DATA_YAML_PATH}")

# Install YOLOv12 and SuperVision

In [ ]:
!pip install -q git+https://github.com/sunsmarterjie/yolov12.git supervision

# Test the base models

### Download example data

In [ ]:
!wget https://media.roboflow.com/notebooks/examples/dog.jpeg

### Run inference

In the example, we're using the `yolov12l.pt` model, but you can experiment with different model sizes by simply swapping out the model name during initialization. Options include `yolov12n.pt`, `yolov12s.pt`, `yolov12m.pt`, `yolov12l.pt`, and `yolov12x.pt`.

In [ ]:
import cv2
from ultralytics import YOLO
import supervision as sv


image_path = f"{HOME}/dog.jpeg"
image = cv2.imread(image_path)

model = YOLO('yolov12l.pt')

results = model(image, verbose=False)[0]
detections = sv.Detections.from_ultralytics(results)

box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()

annotated_image = image.copy()
annotated_image = box_annotator.annotate(scene=annotated_image, detections=detections)
annotated_image = label_annotator.annotate(scene=annotated_image, detections=detections)

sv.plot_image(annotated_image)

# Fine-tune YOLOv12 model

We are now ready to fine-tune our YOLOv12 model. In the code below, we initialize the model using a starting checkpoint—here, we use `yolov12s.yaml`, but you can replace it with any other model (e.g., `yolov12n.pt`, `yolov12m.pt`, `yolov12l.pt`, or `yolov12x.pt`) based on your preference. We set the training to run for 100 epochs in this example; however, you should adjust the number of epochs along with other hyperparameters such as batch size, image size, and augmentation settings (scale, mosaic, mixup, and copy-paste) based on your hardware capabilities and dataset size.

**Note:** **Note that after training, you might encounter a `TypeError: argument of type 'PosixPath' is not iterable error` — this is a known issue, but your model weights will still be saved, so you can safely proceed to running inference.**

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov12m.yaml')
results = model.train(data=DATA_YAML_PATH, epochs=100, patience=10)
# results = model.train(data=DATA_YAML_PATH, epochs=17)

### Save model file into google drive

In [ ]:
import shutil
import os

# Define the source path of the best trained model
source_path = f'{HOME}/runs/detect/train/weights/best.pt'

# Define the destination directory in Google Drive
destination_dir = '/content/drive/MyDrive/trained_models'

# Create the destination directory if it doesn't exist
os.makedirs(destination_dir, exist_ok=True)

# Define the full destination path
destination_path = os.path.join(destination_dir, 'best_yolov12l_bbox5k.pt')

# Copy the model file to Google Drive
shutil.copy(source_path, destination_path)

print(f"Trained model saved to: {destination_path}")

# Evaluate fine-tuned YOLOv12 model

### Inspect Training Output Directory

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!ls {HOME}/runs/detect/train/

### Display Confusion Matrix

In [ ]:
from IPython.display import Image

Image(filename=f'{HOME}/runs/detect/train/confusion_matrix.png', width=1000)

### Display Training Results Plot

In [ ]:
from IPython.display import Image

Image(filename=f'{HOME}/runs/detect/train/results.png', width=1000)

### Load Test Dataset and Display Classes

In [ ]:
import supervision as sv

ds = sv.DetectionDataset.from_yolo(
    images_directory_path=f"{DATASET_PATH}/test/images",
    annotations_directory_path=f"{DATASET_PATH}/test/labels",
    data_yaml_path=DATA_YAML_PATH
)

ds.classes

### Calculate Mean Average Precision (mAP)

In [ ]:
from supervision.metrics import MeanAveragePrecision

model = YOLO(f'/{HOME}/runs/detect/train/weights/best.pt')

predictions = []
targets = []

for _, image, target in ds:
    results = model(image, verbose=False)[0]
    detections = sv.Detections.from_ultralytics(results)

    predictions.append(detections)
    targets.append(target)

map = MeanAveragePrecision().update(predictions, targets).compute()

### Print mAP Scores

In [ ]:
print("mAP 50:95", map.map50_95)
print("mAP 50", map.map50)
print("mAP 75", map.map75)

### Plot mAP Curve

In [ ]:
map.plot()

# Run inference with fine-tuned YOLOv12 model

In [ ]:
import supervision as sv

model = YOLO(f'/{HOME}/runs/detect/train/weights/best.pt')

ds = sv.DetectionDataset.from_yolo(
    images_directory_path=f"{DATASET_PATH}/test/images",
    annotations_directory_path=f"{DATASET_PATH}/test/labels",
    data_yaml_path=DATA_YAML_PATH
)

In [ ]:
import random

i = random.randint(0, len(ds))

image_path, image, target = ds[i]

results = model(image, verbose=False)[0]
detections = sv.Detections.from_ultralytics(results).with_nms()

box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()

annotated_image = image.copy()
annotated_image = box_annotator.annotate(scene=annotated_image, detections=detections)
annotated_image = label_annotator.annotate(scene=annotated_image, detections=detections)

sv.plot_image(annotated_image)

# Call tắt laptop sau khi train xong

In [ ]:
import requests

WEBHOOK_URL = "https://forcepslike-lawanda-semicalcined.ngrok-free.dev/done"  # đổi URL của bạn

try:
    r = requests.post(WEBHOOK_URL)
    print("Đã gửi tín hiệu shutdown về PC!")
except Exception as e:
    print("Không gửi được webhook:", e)